In [ ]:
!pip install -q pandas sklearn cryptography gensim xgboost nltk

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn import datasets as ds
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import re
import csv
import random
import string
import multiprocessing
import itertools
import random
import csv
import statistics
import string
import random
from tensorflow.keras.regularizers import l2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from abc import ABC, abstractmethod

class FFXBase(ABC):

    @abstractmethod
    def encrypt(data, key, tweak):
        pass

    @abstractmethod
    def decrypt(data, key, tweak):
        pass


In [ ]:
from functools import reduce
#from cryptography.hazmat.primitives.ciphers import Cipher, modes
import math

class FFXError(Exception):
    pass


def num_radix(val, radix):
    return reduce(lambda x, c: x * radix + c, val, 0)

def prf(cipher, data):
    # The spec implements cbc mode manually. I'll use the defaule implementation to make use of faster implementations under the hood (in this case OpenSSL)
    # IV = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    encryptor = Cipher(cipher, modes.CBC(bytes([0] * 16))).encryptor()
    ct = encryptor.update(data) + encryptor.finalize()
    return ct[-16:]

def xor_generator(a, b):
    return (x ^ y for x, y in zip(a, b))

def xor(a, b):
    return bytes(xor_generator(a, b))

def num(x):
    return int.from_bytes(x, 'big')

def str_radix_m(x, m, radix):
    if not (0 <= x <= radix**m):
        raise FFXError(f'Invalid range for x. x must be within [0, {radix**m}]')

    X = [0] * m
    for i in range(m - 1, -1, -1):
        X[i] = x % radix
        x = x // radix
    return X

def string_to_numeral_string(string, inverse_alphabet):
    return [inverse_alphabet[x] for x in string]

def numeral_string_to_string(numeral_string, alphabet):
    return ''.join(alphabet[x] for x in numeral_string)

In [ ]:
import math
from itertools import cycle
#from Crypto.Cipher import AES
from cryptography.hazmat.primitives.ciphers import Cipher, modes

class FF1(FFXBase):
    def __init__(self, cipher, radix, maxTlen, messageLenghth):
        self.cipher = cipher
        self.radix = radix

        if self.radix < 2 or self.radix > 2**16:
            raise FFXError(f'The alphabeth length (radix) has to be within [2, {2**16}]')

        self.minlen = messageLenghth[0]
        self.maxlen = messageLenghth[1]

        # 1000000 is defined in the spec as security margin
        if self.radix**self.minlen < 1000000:
            raise FFXError(f'For the given radix, the minimal message length must be at least {math.ceil(math.log2(1000000) / math.log2(self.radix))}')

        if not (2 <= self.minlen <= self.maxlen <= 2**32):
            raise FFXError(f'Requirements for message length not met: 2 <= minlen ({self.minlen}) <= maxlen ({self.maxlen}) <= {2**32}')

        self.maxTlen = maxTlen

    def encrypt(self, data, key: bytes, tweak: bytes):
        print("enc2")
        n = len(data)
        t = len(tweak)
        encryptor = Cipher(self.cipher(key), modes.ECB()).encryptor()
        if not (self.minlen <= n <= self.maxlen):
            raise FFXError(f'Plaintext length invalid. The plaintext length must be between {self.minlen} and {self.maxlen}. Given: {n}')

        if len(tweak) > self.maxTlen:
            raise FFXError(f'Tweak to long. Maximum allowed length: {self.maxTlen}')

        u = n // 2
        v = n - u
        A, B = data[:u], data[u:]
        b = math.ceil(math.ceil(v * math.log2(self.radix)) / 8)
        d = 4 * math.ceil(b / 4) + 4

        P = bytes([1,2,1]) + self.radix.to_bytes(3, 'big') + bytes([10, u % 256]) + n.to_bytes(4, 'big') + t.to_bytes(4, 'big')

        # 10 rounds
        for i in range(10):
            Q = tweak + (0).to_bytes((-t - b - 1) % 16, 'big') + i.to_bytes(1, 'big') + num_radix(B, self.radix).to_bytes(b, 'big')
            R = prf(self.cipher(key), P + Q)
            R_cycle = cycle(R)
            S = (R + b''.join(
                    encryptor.update(xor(R_cycle, bytes(tmp.to_bytes(16, 'big')))) for tmp in range(1, (d // 16) + 1)
                    )
                )[:d]

            y = num(S)

            if i % 2 == 0:
                m = u
            else:
                m = v

            c = (num_radix(A, self.radix) + y) % self.radix**m
            C = str_radix_m(c, m, self.radix)

            A = B
            B = C

        return A + B



    def decrypt(self,data, key, tweak):
        n = len(data)
        t = len(tweak)
        encryptor = Cipher(self.cipher(key), modes.ECB()).encryptor()

        if not (self.minlen <= n <= self.maxlen):
            raise FFXError(f'Ciphertext length invalid. The ciphertext length must be between {self.minlen} and {self.maxlen}. Given: {n}')

        if len(tweak) > self.maxTlen:
            raise FFXError(f'Tweak to long. Maximum allowed length: {self.maxTlen}')

        u = n // 2
        v = n - u
        A, B = data[:u], data[u:]
        b = math.ceil(math.ceil(v * math.log2(self.radix)) / 8)
        d = 4 * math.ceil(b / 4) + 4

        P = bytes([1,2,1]) + self.radix.to_bytes(3, 'big') + bytes([10, u % 256]) + n.to_bytes(4, 'big') + t.to_bytes(4, 'big')

        # 10 rounds
        for i in range(9, -1, -1):
            Q = tweak + (0).to_bytes((-t - b - 1) % 16, 'big') + i.to_bytes(1, 'big') + num_radix(A, self.radix).to_bytes(b, 'big')
            R = prf(self.cipher(key), P + Q)
            R_cycle = cycle(R)
            S = (R + b''.join(
                    encryptor.update(xor(R_cycle, bytes(tmp.to_bytes(16, 'big')))) for tmp in range(1, (d // 16) + 1)
                    )
                )[:d]

            y = num(S)
            if i % 2 == 0:
                m = u
            else:
                m = v

            c = (num_radix(B, self.radix) - y) % self.radix**m
            C = str_radix_m(c, m, self.radix)
            B = A
            A = C

        return A + B

In [ ]:

import unittest

import string
from cryptography.hazmat.primitives.ciphers import algorithms

class TestUtils(unittest.TestCase):
    def test_string_to_numeral_string(self):
        inverse_alphabet = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}
        res = string_to_numeral_string("1234567890", inverse_alphabet)
        self.assertEqual(res, [1, 2, 3, 4, 5, 6, 7, 8, 9, 0])

    def test_string_to_numeral_string_key_error(self):
        inverse_alphabet = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}
        self.assertRaises(KeyError, string_to_numeral_string, "1234567890A", inverse_alphabet)

    def test_numeral_string_to_string(self):
        res = numeral_string_to_string([1, 2, 3, 4, 5, 6, 7, 8, 9, 0], string.digits)
        self.assertEqual(res, "1234567890")

    def test_numeral_string_to_string_key_error(self):
        self.assertRaises(IndexError, numeral_string_to_string, [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 10], string.digits)


    def test_numeral_string_string_numeral(self):
        inverse_alphabet = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}
        alphabet = string.digits
        indata = "1234567890123"
        res = numeral_string_to_string(string_to_numeral_string(indata, inverse_alphabet), alphabet)
        self.assertEqual(res, indata)


    def test_xor(self):
        res = xor([1, 2, 3], [0, 0, 0])
        self.assertEqual(res, bytes([1, 2, 3]))

    def test_xor(self):
        res = xor([1, 2, 3], [1, 2, 3])
        self.assertEqual(res, bytes([0, 0, 0]))

    def test_num(self):
        res = num(bytes([1, 1]))
        self.assertEqual(res, 257)

    def test_num(self):
        res = num(bytes([255, 255]))
        self.assertEqual(res, 0xffff)

    def test_str_radix_m(self):
        self.assertEqual(str_radix_m(559, 4, 12), [0, 3, 10, 7])

    def test_str_radix_m_invalid_range(self):
        self.assertRaises(FFXError, str_radix_m, 559, 2, 2)

    def test_prf(self):
        res = prf(algorithms.AES(b'das ist ein test'), b'Das ist eine sehr geheime Nachricht. Nicht weite')
        self.assertEqual(len(res), 16)
        self.assertEqual(res, bytes([0x66, 0xce, 0x95, 0xe2, 0xaf, 0x0c, 0xc8, 0xfb, 0x82, 0x89, 0x0a, 0x88, 0xd9, 0x6e, 0x18, 0x68]))

    def test_prf_invalid_length(self):
        self.assertRaises(ValueError, prf, algorithms.AES(b'das ist ein test'), b'Das ist eine sehr geheime Nachricht. Nicht weitersagen!')

    def test_num_radix(self):
        res = num_radix([0, 0, 0, 1, 1, 0, 1, 0], 5)
        self.assertEqual(res, 755)

In [ ]:
# from .FF1 import *
# from .FF3_1 import *
# from .utils.FFXBase import FFXBase
# from .utils import string_to_numeral_string, numeral_string_to_string

def encrypt(crypter: FFXBase, plaintext, alphabet, **kwargs):
    # return crypter.encrypt( # encrypt using format preserving encryption
    #         string_to_numeral_string( # convert string to numeral string
    #             plaintext,
    #             {value: key for key, value in enumerate(alphabet)}
    #             ),
    #         **kwargs
    #         )

    return numeral_string_to_string(  # Convert numerals back to original format
        crypter.encrypt( # encrypt using format preserving encryption
            string_to_numeral_string( # convert string to numeral string
                plaintext,
                {value: key for key, value in enumerate(alphabet)}
                ),
            **kwargs
            ),
            alphabet
    )


def decrypt(crypter: FFXBase, ciphertext, alphabet, **kwargs):
    return numeral_string_to_string(  # Convert numerals back to original format
        crypter.decrypt( # encrypt using format preserving encryption
            string_to_numeral_string( # convert string to numeral string
                ciphertext,
                {value: key for key, value in enumerate(alphabet)}
                ),
            **kwargs
            ),
            alphabet
    )

def encode_int_r(n, alphabet, length=0):
    """
    Return a string representation of a number in the given base system for 2..62

    The string is left in a reversed order expected by the calling cryptographic function

    examples:
       encode_int_r(10, hexdigits)
        'A'
    """
    base = len(alphabet)
    if (base > FF1.RADIX_MAX):
        raise ValueError(f"Base {base} is outside range of supported radix 2..{FF1.RADIX_MAX}")

    x = ''
    while n >= base:
        n, b = divmod(n, base)
        x += alphabet[b]
    x += alphabet[n]

    if len(x) < length:
        x = x.ljust(length, alphabet[0])

    return x


def decode_int_r(astring, alphabet):
    """Decode a Base X encoded string into the number

    Arguments:
    - `astring`: The encoded string
    - `alphabet`: The alphabet to use for decoding
    """
    strlen = len(astring)
    base = len(alphabet)
    num = 0

    idx = 0
    try:
        for char in reversed(astring):
            power = (strlen - (idx + 1))
            num += alphabet.index(char) * (base ** power)
            idx += 1
    except ValueError:
        raise ValueError(f'char {char} not found in alphabet {alphabet}')

    return num



In [ ]:
import numpy as np
from os import urandom
import ast
import string
import random
import sys

def generate_diff(p0):
    x = ast.literal_eval(p0)

    # input difference = 0x08ㅌ
    #x = x ^ 0x08
    x = x ^ 0x07


    temp = (x & 0xff) # 2자리
    p1 = chr(temp)  #1자리


    temp = ((x >> 8) & 0xff)
    temp = chr(temp)

    p1 = temp + p1

    temp = ((x >> 16) & 0xff)
    temp = chr(temp)
    p1 = temp + p1

    temp = ((x >> 24) & 0xff)
    temp = chr(temp)
    p1 = temp + p1

    # temp = ((x >> 32) & 0xff)
    # temp = chr(temp)
    # p1 = temp + p1

    # temp = ((x >> 40) & 0xff)
    # temp = chr(temp)
    # p1 = temp + p1

    return p1

key = bytes([0x2B, 0x7E, 0x15, 0x16, 0x28, 0xAE, 0xD2, 0xA6, 0xAB, 0xF7, 0x15, 0x88, 0x09, 0xCF, 0x4F, 0x3C])
tweak = b''
dy = string.printable
BASE62_LEN = len(dy)
radix = BASE62_LEN
maxTlen = 10
minLen = 4
maxLen = 10


c = FF1(algorithms.AES, radix, maxTlen,(minLen, maxLen))

X = []
n = 800000

def Append(k, text):
    for i in range(0, len(text)):
        X[k].append(list(text)[i])

def Append2(k, text):
        X[k].append(text)

#baseline training data generator


l = 0


hex_p0= np.zeros(shape=(n,),dtype=int)
hex_p1= np.zeros(shape=(n,),dtype=int)
hex_p2= np.zeros(shape=(n,),dtype=int)

hex_c0= np.zeros(shape=(n,),dtype=int)
hex_c1= np.zeros(shape=(n,),dtype=int)
hex_c2= np.zeros(shape=(n,),dtype=int)

hex_c0c1= np.zeros(shape=(n,),dtype=int)
hex_c0c2= np.zeros(shape=(n,),dtype=int)


Y = np.ones(shape=(n,)) # cipher
i=0
while(i != n//2):

  Y[i] = 0 # random
  print(i)

  _LENGTH = 4
  #string_pool = string.digits + string.ascii_lowercase
  string_pool = string.digits #string.digits
  plaintext = ""
  plaintext_ran = ""
  for j in range(_LENGTH):
      plaintext = plaintext + random.choice(string_pool)
      plaintext_ran = plaintext_ran + random.choice(string_pool)

  #hex_p0[i] = int(plaintext.hex(), base=16)
  # bin_p0 = bin (int(hex_p0, base=16))
  # def encrypt(self, data, key: bytes, tweak: bytes):
  # def encrypt(crypter: FFXBase, plaintext, alphabet, **kwargs):
      # return numeral_string_to_string(  # Convert numerals back to original format
      #   crypter.encrypt(string_to_numeral_string(plaintext,{value: key for key, value in enumerate(alphabet)}),**kwargs),alphabet)
  ciphertext = encrypt(c, plaintext, dy, tweak=tweak, key=key)

  c0= ciphertext.encode('utf-8')
  hex_c0[i] = int(c0.hex(), base = 16)

  plaintext0 = plaintext.encode('utf-8')
  temp = plaintext0.hex()
  temp = '0x' + temp
  temp = generate_diff(temp)

  try:
    ciphertext = encrypt(c, temp, dy, tweak=tweak, key=key)
   # ciphertext = numeral_string_to_string(ciphertext,string_pool)
  except:
    continue
  c0= ciphertext.encode('utf-8')

  hex_c2[i] = int(c0.hex(), base = 16)

  plaintext = plaintext.encode('utf-8')
  temp = temp.encode('utf-8')
  hex_p0[i] = int(plaintext.hex(), base=16)
  hex_p2[i] = int(temp.hex(), base=16)

  ciphertext = encrypt(c, plaintext_ran, dy, tweak=tweak, key=key)
  c0= ciphertext.encode('utf-8')

  hex_c1[i] = int(c0.hex(), base = 16)
  plaintext_ran = plaintext_ran.encode('utf-8')
  hex_p1[i] = int(plaintext_ran.hex(), base=16)


  print(hex_c0[i])
  print(hex_c1[i])
  print(hex_c2[i])
  print(hex_p0[i])
  print(hex_p1[i])
  print(hex_p2[i])
  print(type(hex_p1[i]))



  binc0= str(bin(hex_c0[i])).split('0b')[1].zfill(32)
  binc1= str(bin(hex_c1[i])).split('0b')[1].zfill(32)
  binc2= str(bin(hex_c2[i])).split('0b')[1].zfill(32)
  # print("hello")
  # print(len(binc0))

  X.append([l])
  Append(l,binc0)
  Append(l,binc1)
  Append2(l, Y[i])
  l = l+1


  X.append([l])
  Append(l,binc0)
  Append(l,binc2)
  Append2(l, Y[(n//2)+i])

  l = l+1
  i=i+1
  print("hi")

  #if(i%1000 == 0):

0
enc2
enc2
enc2
1617316664
577330805
1881763142
842282290
876098616
842282298
<class 'numpy.int64'>
hi
1
enc2
enc2
enc2
743793752
556360039
1429892363
892875313
909719093
892875321
<class 'numpy.int64'>
hi
2
enc2
enc2
enc2
1061108285
1850549558
2033852455
876163121
809056054
876163129
<class 'numpy.int64'>
hi
3
enc2
enc2
enc2
1193885742
1801878344
1953782563
842347577
909325876
842347569
<class 'numpy.int64'>
hi
4
enc2
enc2
enc2
1395096136
875253558
1850032966
909129784
875640371
909129776
<class 'numpy.int64'>
hi


In [ ]:
print(X[0])
print(len(X[0]))

[0, '0', '1', '1', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '1', '0', '0', '1', '0', '0', '1', '0', '1', '1', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '0', '1', '0', '1', '0', '1', '1', '1', '1', '0', '0', '1', '1', '1', '0', '1', '0', '1', 0.0]
66


In [ ]:
f = open("/content/drive/MyDrive/ff3ds/test.csv", "wt", newline="")
csvwriter = csv.writer(f)
for i in range(0, n):
    csvwriter.writerow(X[i])

print("end")
f.close()


end


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/ff3ds/test.csv",header=None).values

labels = []
datas = []
n_bit = 32
div = 2*n_bit +1


#formatting
for i in range(0,len(data)):
  labels.append(data[i][div:])
  datas.append(data[i][1:div])

datas = np.asarray(datas)
labels = np.asarray(labels)

#print(datas)

x_train, x_val, y_train, y_val = train_test_split(datas, labels, test_size=0.4)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.05)

print(len(x_train))
print(len(x_val))
print(len(x_test))
print(y_test)


5
4
1
[[0.]]


In [ ]:
inp = tf.keras.layers.Input(shape=(64, ))

x = tf.keras.layers.Dense(128, activation = 'relu',kernel_regularizer=l2(0.001))(inp)
shortcut = x
for i in range(2):
    x = tf.keras.layers.Dense(128, activation = 'relu',kernel_regularizer=l2(0.001))(shortcut)
    x = tf.keras.layers.Dense(128, activation = 'relu',kernel_regularizer=l2(0.001))(x)
    shortcut = tf.keras.layers.Add()([shortcut, x])

out = tf.keras.layers.Dense(1, activation = 'sigmoid')(shortcut)
model = tf.keras.Model(inputs=inp, outputs=out)

model.summary()

s = 100 * len(x_train) // 8

lr = keras.optimizers.schedules.ExponentialDecay(0.0001, s, 0.001)
opt = keras.optimizers.Adam(lr)

model.compile(optimizer=opt,
              loss='binary_crossentropy', #binary_crossentropy
              metrics=['accuracy'])

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 dense_12 (Dense)               (None, 128)          8320        ['input_3[0][0]']                
                                                                                                  
 dense_13 (Dense)               (None, 128)          16512       ['dense_12[0][0]']               
                                                                                                  
 dense_14 (Dense)               (None, 128)          16512       ['dense_13[0][0]']               
                                                                                            

In [ ]:
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs=30, batch_size = 32) # 100, 32

Epoch 1/30
7125/7125 [==============================] - 42s 6ms/step - loss: 0.9045 - accuracy: 0.5218 - val_loss: 0.7699 - val_accuracy: 0.5330
Epoch 2/30
7125/7125 [==============================] - 39s 6ms/step - loss: 0.7346 - accuracy: 0.5383 - val_loss: 0.7144 - val_accuracy: 0.5409
Epoch 3/30
7125/7125 [==============================] - 39s 5ms/step - loss: 0.7059 - accuracy: 0.5540 - val_loss: 0.7007 - val_accuracy: 0.5603
Epoch 4/30
7125/7125 [==============================] - 40s 6ms/step - loss: 0.6938 - accuracy: 0.5782 - val_loss: 0.6925 - val_accuracy: 0.5805
Epoch 5/30
7125/7125 [==============================] - 39s 5ms/step - loss: 0.6827 - accuracy: 0.5998 - val_loss: 0.6856 - val_accuracy: 0.5958
Epoch 6/30
7125/7125 [==============================] - 40s 6ms/step - loss: 0.6727 - accuracy: 0.6181 - val_loss: 0.6760 - val_accuracy: 0.6142
Epoch 7/30
7125/7125 [==============================] - 41s 6ms/step - loss: 0.6622 - accuracy: 0.6356 - val_loss: 0.6668 - val_ac

In [ ]:
predicted_labels = model.predict(np.array(x_test))

res = np.array(predicted_labels > 0.5, dtype = int)

res = res.reshape((len(x_test)))
y_test = y_test.reshape((len(y_test)))

total = 0

for i in range(len(res)):
    if res[i] == y_test[i]:
        total += 1

print(total/len(x_test))

375/375 [==============================] - 1s 2ms/step
0.761


In [ ]:
import unittest
#from FFX import *
import string
from cryptography.hazmat.primitives.ciphers import algorithms

# https://csrc.nist.gov/CSRC/media/Projects/Cryptographic-Standards-and-Guidelines/documents/examples/FF1samples.pdf
class TestFF1(unittest.TestCase):
    # AES 128 Bit
    def test_FF1_saple1(self):
        key = bytes([0x2B, 0x7E, 0x15, 0x16, 0x28, 0xAE, 0xD2, 0xA6, 0xAB, 0xF7, 0x15, 0x88, 0x09, 0xCF, 0x4F, 0x3C])
        radix = 10
        maxTlen = 10
        minLen = 6
        maxLen = 10
        tweak = b''
        pt = '0123456789'
        expected = '2433477484'
        alphabet = string.digits
        cipher = FF1(algorithms.AES, radix, maxTlen, (minLen, maxLen))

        ct = encrypt(cipher, pt, alphabet, tweak=tweak, key=key)
        print(ct)
        self.assertEqual(len(ct), len(pt))
        self.assertEqual(ct, expected)

        _pt = decrypt(cipher, ct, alphabet, tweak=tweak, key=key)
        print(pt)
        self.assertEqual(len(_pt), len(pt))
        self.assertEqual(pt, _pt)


    def test_FF1_saple2(self):
        key = bytes([0x2B, 0x7E, 0x15, 0x16, 0x28, 0xAE, 0xD2, 0xA6, 0xAB, 0xF7, 0x15, 0x88, 0x09, 0xCF, 0x4F, 0x3C])
        radix = 10
        maxTlen = 10
        minLen = 6
        maxLen = 10
        tweak = bytes([0x39, 0x38, 0x37, 0x36, 0x35, 0x34, 0x33, 0x32, 0x31, 0x30])
        pt = '0123456789'
        expected = '6124200773'
        alphabet = string.digits
        cipher = FF1(algorithms.AES, radix, maxTlen, (minLen, maxLen))

        ct = encrypt(cipher, pt, alphabet, tweak=tweak, key=key)
        print(ct)
        self.assertEqual(len(ct), len(pt))
        self.assertEqual(ct, expected)

        _pt = decrypt(cipher, ct, alphabet, tweak=tweak, key=key)
        print(pt)
        self.assertEqual(len(_pt), len(pt))
        self.assertEqual(pt, _pt)

    def test_FF1_saple3(self):
        key = bytes([0x2B, 0x7E, 0x15, 0x16, 0x28, 0xAE, 0xD2, 0xA6, 0xAB, 0xF7, 0x15, 0x88, 0x09, 0xCF, 0x4F, 0x3C])
        radix = 36
        maxTlen = 11
        minLen = 6
        maxLen = 20
        tweak = bytes([0x37, 0x37, 0x37, 0x37, 0x70, 0x71, 0x72, 0x73, 0x37, 0x37, 0x37])
        pt = '0123456789abcdefghi'
        expected = 'a9tv40mll9kdu509eum'
        alphabet = '0123456789abcdefghijklmnopqrstuvwxyz'
        cipher = FF1(algorithms.AES, radix, maxTlen, (minLen, maxLen))

        ct = encrypt(cipher, pt, alphabet, tweak=tweak, key=key)
        self.assertEqual(len(ct), len(pt))
        self.assertEqual(ct, expected)

        _pt = decrypt(cipher, ct, alphabet, tweak=tweak, key=key)
        self.assertEqual(len(_pt), len(pt))
        self.assertEqual(pt, _pt)


    def test_FF1_saple4(self):
        key = bytes([0x2B, 0x7E, 0x15, 0x16, 0x28, 0xAE, 0xD2, 0xA6, 0xAB, 0xF7, 0x15, 0x88, 0x09, 0xCF, 0x4F, 0x3C])
        radix = 36
        maxTlen = 11
        minLen = 6
        maxLen = 20
        tweak = bytes([0x37, 0x37, 0x37, 0x37, 0x70, 0x71, 0x72, 0x73, 0x37, 0x37, 0x37])
        pt = '0123456789abcdefghi'
        expected = 'a9tv40mll9kdu509eum'
        alphabet = '0123456789abcdefghijklmnopqrstuvwxyz'
        cipher = FF1(algorithms.AES, radix, maxTlen, (minLen, maxLen))

        ct = encrypt(cipher, pt, alphabet, tweak=tweak, key=key)
        self.assertEqual(len(ct), len(pt))
        self.assertEqual(ct, expected)

        _pt = decrypt(cipher, ct, alphabet, tweak=tweak, key=key)
        self.assertEqual(len(_pt), len(pt))
        self.assertEqual(pt, _pt)






    # AES 192Bit
    def test_FF1_saple5(self):
        key = bytes([0x2B, 0x7E, 0x15, 0x16, 0x28, 0xAE, 0xD2, 0xA6, 0xAB, 0xF7, 0x15, 0x88, 0x09, 0xCF, 0x4F, 0x3C, 0xEF, 0x43, 0x59, 0xD8, 0xD5, 0x80, 0xAA, 0x4F])
        radix = 10
        maxTlen = 10
        minLen = 6
        maxLen = 10
        tweak = b''
        pt = '0123456789'
        expected = '2830668132'
        alphabet = string.digits
        cipher = FF1(algorithms.AES, radix, maxTlen, (minLen, maxLen))

        ct = encrypt(cipher, pt, alphabet, tweak=tweak, key=key)
        self.assertEqual(len(ct), len(pt))
        self.assertEqual(ct, expected)

        _pt = decrypt(cipher, ct, alphabet, tweak=tweak, key=key)
        self.assertEqual(len(_pt), len(pt))
        self.assertEqual(pt, _pt)


    def test_FF1_saple6(self):
        key = bytes([0x2B, 0x7E, 0x15, 0x16, 0x28, 0xAE, 0xD2, 0xA6, 0xAB, 0xF7, 0x15, 0x88, 0x09, 0xCF, 0x4F, 0x3C, 0xEF, 0x43, 0x59, 0xD8, 0xD5, 0x80, 0xAA, 0x4F])
        radix = 36
        maxTlen = 16
        minLen = 6
        maxLen = 20
        tweak = bytes([0x37, 0x37, 0x37, 0x37, 0x70, 0x71, 0x72, 0x73, 0x37, 0x37, 0x37])
        pt = '0123456789abcdefghi'
        expected = 'xbj3kv35jrawxv32ysr'
        alphabet = '0123456789abcdefghijklmnopqrstuvwxyz'
        cipher = FF1(algorithms.AES, radix, maxTlen, (minLen, maxLen))

        ct = encrypt(cipher, pt, alphabet, tweak=tweak, key=key)
        self.assertEqual(len(ct), len(pt))
        self.assertEqual(ct, expected)

        _pt = decrypt(cipher, ct, alphabet, tweak=tweak, key=key)
        self.assertEqual(len(_pt), len(pt))
        self.assertEqual(pt, _pt)







    # AES 256Bit
    def test_FF1_saple7(self):
        key = bytes([0x2B, 0x7E, 0x15, 0x16, 0x28, 0xAE, 0xD2, 0xA6, 0xAB, 0xF7, 0x15, 0x88, 0x09, 0xCF, 0x4F, 0x3C, 0xEF, 0x43, 0x59, 0xD8, 0xD5, 0x80, 0xAA, 0x4F, 0x7F, 0x03, 0x6D, 0x6F, 0x04, 0xFC, 0x6A, 0x94])
        radix = 10
        maxTlen = 10
        minLen = 6
        maxLen = 10
        tweak = b''
        pt = '0123456789'
        expected = '6657667009'
        alphabet = string.digits
        cipher = FF1(algorithms.AES, radix, maxTlen, (minLen, maxLen))

        ct = encrypt(cipher, pt, alphabet, tweak=tweak, key=key)
        self.assertEqual(len(ct), len(pt))
        self.assertEqual(ct, expected)

        _pt = decrypt(cipher, ct, alphabet, tweak=tweak, key=key)
        self.assertEqual(len(_pt), len(pt))
        self.assertEqual(pt, _pt)


    def test_FF1_saple8(self):
        key = bytes([0x2B, 0x7E, 0x15, 0x16, 0x28, 0xAE, 0xD2, 0xA6, 0xAB, 0xF7, 0x15, 0x88, 0x09, 0xCF, 0x4F, 0x3C, 0xEF, 0x43, 0x59, 0xD8, 0xD5, 0x80, 0xAA, 0x4F, 0x7F, 0x03, 0x6D, 0x6F, 0x04, 0xFC, 0x6A, 0x94])
        radix = 10
        maxTlen = 16
        minLen = 6
        maxLen = 20
        tweak = bytes([0x39, 0x38, 0x37, 0x36, 0x35, 0x34, 0x33, 0x32, 0x31, 0x30])
        pt = '0123456789'
        expected = '1001623463'
        alphabet = '0123456789'
        cipher = FF1(algorithms.AES, radix, maxTlen, (minLen, maxLen))

        ct = encrypt(cipher, pt, alphabet, tweak=tweak, key=key)
        self.assertEqual(len(ct), len(pt))
        self.assertEqual(ct, expected)

        _pt = decrypt(cipher, ct, alphabet, tweak=tweak, key=key)
        self.assertEqual(len(_pt), len(pt))
        self.assertEqual(pt, _pt)



    def test_FF1_saple6(self):
        key = bytes([0x2B, 0x7E, 0x15, 0x16, 0x28, 0xAE, 0xD2, 0xA6, 0xAB, 0xF7, 0x15, 0x88, 0x09, 0xCF, 0x4F, 0x3C, 0xEF, 0x43, 0x59, 0xD8, 0xD5, 0x80, 0xAA, 0x4F, 0x7F, 0x03, 0x6D, 0x6F, 0x04, 0xFC, 0x6A, 0x94])
        radix = 36
        maxTlen = 16
        minLen = 6
        maxLen = 20
        tweak = bytes([0x37, 0x37, 0x37, 0x37, 0x70, 0x71, 0x72, 0x73, 0x37, 0x37, 0x37])
        pt = '0123456789abcdefghi'
        expected = 'xs8a0azh2avyalyzuwd'
        alphabet = '0123456789abcdefghijklmnopqrstuvwxyz'
        cipher = FF1(algorithms.AES, radix, maxTlen, (minLen, maxLen))

        ct = encrypt(cipher, pt, alphabet, tweak=tweak, key=key)
        self.assertEqual(len(ct), len(pt))
        self.assertEqual(ct, expected)

        _pt = decrypt(cipher, ct, alphabet, tweak=tweak, key=key)
        self.assertEqual(len(_pt), len(pt))
        self.assertEqual(pt, _pt)


In [ ]:
test = TestFF1()
test.test_FF1_saple2()
#TestFF1.test_FF1_saple1()

enc2
6124200773
0123456789
